In [1]:
from huggingface_hub import notebook_login, interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token can be pasted using 'Right-Click'.


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  n


Token is valid (permission: read).
Your token has been saved to C:\Users\katko\.cache\huggingface\token
Login successful


In [2]:
from transformers.utils import send_example_telemetry

send_example_telemetry("token_classification_notebook", framework="pytorch")

In [3]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Preprocessing

In [4]:
from preprocessing import extract_data_from_conll_extended
extract_data_from_conll_extended("data/en_ewt-up-test.conllu", "dummy_train.json")

In [5]:
from datasets import load_dataset, load_metric
datasets = load_dataset('json', data_files={'train': 'dummy_train.json', 'test': 'dummy_test.json'})
datasets

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['predicate_position', 'arguments', 'predicate', 'word'],
        num_rows: 4799
    })
    test: Dataset({
        features: ['predicate_position', 'arguments', 'predicate', 'word'],
        num_rows: 12
    })
})

In [6]:
datasets["train"][0]

{'predicate_position': 4,
 'arguments': ['_', '_', 'ARG1', 'V', '_', 'ARG2', '_'],
 'predicate': 'morph.01',
 'word': ['What', 'if', 'Google', 'Morphed', 'Into', 'GoogleOS', '?']}

In [7]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(datasets["train"])

,predicate_position,arguments,predicate,word
0,5,"[_, _, ARG1, _, V, ARG2]",be.01,"[Cleanest, guesthouse, i, have, been, to]"
1,9,"[_, _, _, ARG1, _, _, ARG0, ARGM-MOD, V, _]",provide.01,"[Grateful, for, any, help, or, suggestions, you, could, provide, .]"
2,13,"[_, _, _, _, _, _, _, _, _, _, ARG0, _, V, _, _, ARG1, _, _, _, _, ARGM-COM, _]",participate.01,"[Az, -, Zaman, :, The, Association, of, Muslim, Scholars, forbade, Iraqis, to, participate, in, the, attack, on, Fallujah, with, the, Americans, .]"
3,6,"[_, _, _, _, _, V, _, _, _, ARG1, _, _, _, _, _, _, _, _, _, ARGM-ADJ, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _]",deal.01,"[Clinton, had, worked, out, a, deal, with, Pakistani, Prime, Minister, Nawaz, Sharif, in, summer, of, 1999, that, would, have, allowed, the, US, to, send, a, Special, Ops, team, in, after, Bin, Laden, in, Qandahar, ,, based, from, Pakistan, .]"
4,17,"[ARG1, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, V, C-ARG1, _, _, _, _, _, _, _, _, _, _]",seem.01,"[They, want, to, use, LTS, to, tie, into, NASA, 's, vision, for, Space, Exploration, ,, and, seem, pleased, that, the, White, House, is, moving, in, that, direction, .]"
5,6,"[_, _, _, _, _, V, _, _]",copy.01,"[I, shall, send, you, a, copy, today, .]"
6,13,"[_, _, _, _, _, _, _, _, _, _, ARG1, _, V, _]",great.01,"[The, price, they, gave, was, good, so, I, said, hey, this, seems, great, .]"
7,16,"[_, _, _, _, _, _, _, _, _, _, _, ARG1, _, _, _, V, _, _, ARG2, _, _, _]",be.01,"[the, economy, is, down, and, when, enron, collapses, ,, the, energy, industry, is, going, to, be, in, a, world, of, hurt, .]"
8,3,"[_, ARG1, V, ARG2, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _, _]",be.01,"[Think, that, was, bad, -, by, December, of, '74, the, Dow, had, sunk, to, a, bear, market, low, of, 577, .]"
9,3,"[_, ARG0, V, _, _, ARG1, _, _, _, _, _, _, _, _, _, _]",want.01,"[If, you, want, a, CD, copy, of, this, web, site, ,, give, me, a, yell, .]"


In [9]:
datasets["train"][0]

{'predicate_position': 4,
 'arguments': ['_', '_', 'ARG1', 'V', '_', 'ARG2', '_'],
 'predicate': 'morph.01',
 'word': ['What', 'if', 'Google', 'Morphed', 'Into', 'GoogleOS', '?']}

### Tokenization

In [10]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [11]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [12]:
def tokenize(instance):
    def align(tokenized_input, labels):
        word_ids = tokenized_input.word_ids()
        aligned_labels = ['_'] * len(word_ids) 
        label_index = 0 
        for i, word_id in enumerate(word_ids):
            try:
                if word_id is None:
                    aligned_labels[i] = '[PAD]'
                    continue 
                original_label = labels[word_id]
                if original_label == '_':
                    continue 
                if i == 0 or word_id != word_ids[i-1]:
                    prefix = ''
                else:
                    prefix = ''
                aligned_labels[i] = f'{prefix}{original_label}'
            except:
                pass
    
        return aligned_labels
    sentence = instance["word"]
    labels = instance["arguments"]
    tokenized_input = tokenizer(sentence, is_split_into_words=True)
    predicate = instance['predicate']
    predicate = tokenizer(predicate)
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
    aligned_labels = align(tokenized_input, labels)
    tokenized_input['input_ids'].extend(predicate['input_ids'][1:]), tokenized_input['attention_mask'].extend(predicate['attention_mask'][1:]), aligned_labels.extend(['[PAD]' for i in range(len(predicate['input_ids'][1:]))])
    tokenized_input['labels'] = aligned_labels
    
    return tokenized_input


In [13]:
print(len(datasets['train']))
training_set = [tokenize(instance) for instance in datasets['train']]
test_set = [tokenize(instance) for instance in datasets['test']]


4799


In [14]:
sample = training_set[0]
print(sample)

{'input_ids': [101, 2054, 2065, 8224, 22822, 8458, 2098, 2046, 8224, 2891, 1029, 102, 22822, 8458, 1012, 5890, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': ['[PAD]', '_', '_', 'ARG1', 'V', 'V', 'V', '_', 'ARG2', 'ARG2', '_', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']}


In [56]:
from datasets import Dataset, Features, Sequence, Value

features = Features({
    "input_ids": Sequence(feature=Value(dtype='int64')),
    "attention_mask": Sequence(feature=Value(dtype='int64')),
    "labels": Sequence(feature=Value(dtype='int64')),
})

In [74]:
label_list = list(set(label for instance in training_set for label in instance['labels']))
label_dict = {label:int(i) for i, label in enumerate(list(label_list))}
label_dict['[PAD]'] = -100
for instance in training_set:
    instance['labels'] = [int(label_dict[label]) for label in instance['labels']]
for instance in test_set:
    instance['labels'] = [int(label_dict[label]) for label in instance['labels']]

In [75]:
dataset_train = Dataset.from_dict({"input_ids": [item['input_ids'] for item in training_set],
                             "attention_mask": [item['attention_mask'] for item in training_set],
                             "labels": [item['labels'] for item in training_set]}, features=features)

dataset_test = Dataset.from_dict({"input_ids": [item['input_ids'] for item in test_set],
                             "attention_mask": [item['attention_mask'] for item in test_set],
                             "labels": [item['labels'] for item in test_set]}, features=features)

In [76]:
#print(tokenized_input.word_ids())
print(dataset_train[0])

{'input_ids': [101, 2054, 2065, 8224, 22822, 8458, 2098, 2046, 8224, 2891, 1029, 102, 22822, 8458, 1012, 5890, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [0, 36, 36, 2, 20, 20, 20, 36, 30, 30, 36, 0, 0, 0, 0, 0, 0]}


In [77]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [78]:
dataset_train[0]

{'input_ids': [101,
  2054,
  2065,
  8224,
  22822,
  8458,
  2098,
  2046,
  8224,
  2891,
  1029,
  102,
  22822,
  8458,
  1012,
  5890,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [0, 36, 36, 2, 20, 20, 20, 36, 30, 30, 36, 0, 0, 0, 0, 0, 0]}

In [79]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [80]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [81]:
metric = load_metric("seqeval")
metric = load_metric("seqeval")


C:\Users\katko\miniconda3\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
C:\Users\katko\miniconda3\Lib\site-packages\datasets\load.py:756: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metri

In [82]:
labels = list(label_list)
metric.compute(predictions=[labels], references=[labels])

C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 2 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 3 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeling.py:171: UserWarning: 4 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\sequence_labeli

{'0': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 4},
 '1': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 4},
 '2': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 4},
 '3': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 4},
 '4': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 4},
 '5': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 4},
 '6': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 '7': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 '8': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 '9': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 3},
 '_': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [83]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [84]:
# from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

trainer = Trainer(
    model,
    args,
    train_dataset=dataset_train,
    eval_dataset=dataset_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.182814,0.723077,0.618421,0.666667,0.956667


C:\Users\katko\miniconda3\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
dataset_test[0]